# Eager Execution 튜토리얼: 기초

본 노트북은 텐서플로우의 eager execution의 기능을 소개하기 위한 기초 자료입니다. 다음과 같은 내용을 포함하고 있습니다:
* 필요한 패키지 불러오기
* eager execution 활성화
* TensorFlow 텐서와 변수를 만들고 사용하기 
* TensorFlow와 상호작용하며 사용하기
* eager execution 활성화 상태에서 GPU 사용하기 

본 노트북은 그레디언트와 같은 모델링 토픽은 다루고 있지 않습니다.

# Step 1: Eager 불러오기
eager execution을 위해서 다음과 같이 import 하세요:

In [ ]:
# TensorFlow 불러오기
import tensorflow as tf

# tfe 모듈은 즉시 실행 모드와 그래프 실행 모드 양쪽다 동작하는 다양한 기능들을 포함하고 있습니다.
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

In [4]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))
print("")

x = tf.constant(2)
y = tf.constant(3)
print(x * y + 1)

# 대부분의 TensorFlow 연산은 eager execution에서 즉시 사용가능합니다.
# 다음과 같이 즉시 값을 반환해줍니다.
print(tf.contrib.signal.hamming_window(x * y + 1))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(aGVsbG8gd29ybGQ, shape=(), dtype=string)

tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(
[0.08000001 0.31000003 0.77000004 1.         0.77       0.30999985
 0.08000001], shape=(7,), dtype=float32)


In [5]:
import numpy as np

ones = np.ones([3, 3])

print("1로 구성된 numpy 3x3행렬은:")
print(ones)
print("")

print("42를 곱하면:")
print(tf.multiply(ones, 42))

numpy 3x3 matrix of 1s:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

Multiplied by 42:
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)


In [6]:
x = tf.get_variable(name="x", shape=[], dtype=tf.float32, initializer=tf.zeros_initializer)

In [7]:
# This does NOT print the Variable's actual value:
print("Printing a TensorFlow Variable:")
print(x)
print("")

# A TensorFlow variable represents a reference to a tensor.
# The `read_value()` method provides access to the current value of the
# variable. Tensorflow Variables are automatically initialized according to the
# semantics defined in tf.get_variable().
print("Printing a TensorFlow Variable's value using .read_value():")
print(x.read_value())
print("")

print("Printing a TensorFlow Variable's value using .read_value().numpy():")
print(x.read_value().numpy())

Printing a TensorFlow Variable:
<tf.Variable 'x:0' shape=() dtype=float32, numpy=0.0>

Printing a TensorFlow Variable's value using .read_value():
tf.Tensor(0.0, shape=(), dtype=float32)

Printing a TensorFlow Variable's value using .read_value().numpy():
0.0


In [8]:
x.assign(42)
print(x.read_value())

x.assign_add(3)
print(x.read_value())

tf.Tensor(42.0, shape=(), dtype=float32)
tf.Tensor(45.0, shape=(), dtype=float32)


In [9]:
print(x + 3)

# This code will broadcast the value across the list of numbers:
print(x * [1, 2, 4])

tf.Tensor(48.0, shape=(), dtype=float32)
tf.Tensor([ 45.  90. 180.], shape=(3,), dtype=float32)


In [10]:
vector = tf.constant([10.0, 20.0, 30.0, 40.0])

In [11]:
# Works, because the values of `begin` and `size` (the 2nd and 3rd input
# arguments) are within the bound of `vector`.
print(tf.slice(vector, [1], [3]))

tf.Tensor([20. 30. 40.], shape=(3,), dtype=float32)


In [12]:
# The following does NOT work, because the value of `size` (the 3rd
# argument) causes the indices to go out of the bounds of `vector`. The
# error is raised immediately.
try:
  print(tf.slice(vector, [1], [4]))
except tf.OpError as e:
  print("Caught error: %s" % e)

Caught error: Expected size[0] in [0, 3], but got 4 [Op:Slice]


In [13]:
# The example code from here on will work only if your notebook
# is running on a machine with a functional CUDA GPU. The following
# line checks that.
is_gpu_available = tfe.num_gpus() > 0

# Create some Tensors
SIZE = 1000
cpu_tensor = tf.random_normal([SIZE, SIZE])

if is_gpu_available:
  gpu_tensor = cpu_tensor.gpu()
else:
  print("GPU not available.")

In [14]:
# Time a CPU-based matrix multiplication

print("Time to conduct matmul on CPU:")
%time tf.matmul(cpu_tensor, cpu_tensor)

Time to conduct matmul on CPU:
CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 64.9 ms


<tf.Tensor: id=104, shape=(1000, 1000), dtype=float32, numpy=
array([[ 22.38106  ,   5.633296 ,  13.299532 , ...,  31.061148 ,
         -7.768629 , -48.17466  ],
       [ 43.470367 ,   1.5255723, -32.61236  , ..., -23.339508 ,
        -32.330822 , -33.298195 ],
       [  6.201656 ,  45.583122 ,  29.281898 , ..., -72.03731  ,
         15.40768  ,  25.029882 ],
       ...,
       [ 25.847857 ,  38.240105 , -41.389606 , ..., -18.695236 ,
        -26.908352 ,  49.71857  ],
       [-10.576298 , -13.079933 ,  -6.115379 , ..., -37.376156 ,
        -40.50897  , -16.68614  ],
       [ 18.368586 , -42.574005 , -20.013565 , ...,  15.061981 ,
         -5.717218 , -28.637989 ]], dtype=float32)>

In [15]:
# Time GPU-based matrix multiplications.

if is_gpu_available:
  # First use of the GPU will be slow:
  print("Time to conduct first matmul on GPU:")
  %time tf.matmul(gpu_tensor, gpu_tensor)
  print()

  # Subsequent uses are much faster:
  print("Time to conduct second matmul on GPU:")
  %time tf.matmul(gpu_tensor, gpu_tensor)

Time to conduct first matmul on GPU:
CPU times: user 92 ms, sys: 60 ms, total: 152 ms
Wall time: 667 ms
()
Time to conduct second matmul on GPU:
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 143 µs


In [16]:
# Second timing demo for GPUs, after it has been used once:

cpu_tensor = tf.random_normal([SIZE, SIZE])
print("Time to conduct CPU matmul:")
%time tf.matmul(cpu_tensor, cpu_tensor)
print()

if is_gpu_available:
  gpu_tensor = cpu_tensor.gpu()
  print("Time to conduct GPU matmul:")
  %time tf.matmul(gpu_tensor, gpu_tensor)

Time to conduct CPU matmul:
CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 48.5 ms
()
Time to conduct GPU matmul:
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 173 µs
